# PDF 문서를 image로 변환 후 요약하는 샘플 코드

In [1]:
!pip install PyMuPDF Pillow boto3

## 1. 매서드 작성

In [11]:
import fitz  # PyMuPDF
import boto3
import base64
from PIL import Image
import io
import json

In [12]:
# 1. PDF를 이미지로 변환
def pdf_to_images(pdf_path):
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    return images

In [13]:
# 2. 이미지를 Base64로 인코딩
def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

In [14]:
# 3. Amazon Bedrock 클라이언트 설정
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # 예: 'us-east-1'
)

In [18]:
# 4. Claude 모델에 이미지 전송 및 응답 받기
def get_claude_response(image_base64):
    
    prompt = """
    다음 <instruction></instruction>에 따라 정리 해주세요.
    <instruction>
    1. 중요 정보나 객관적 데이터를 누락하지 말고 반드시 포함하세요.
    2. 테이블이나 차트와 같은 부분에 대해서도 자세한 내용을 포함하세요.
    3. 타이틀이 있는 내용은 타이틀 이름도 반드시 포함하세요.
    4. 자세하게 정리하세요.
    </instruction>
    """
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    #테스트 환경에 Claude 3.5 사용이 불가합니다. 고객 분들은 아래 modelId를 Claude 3.5 Sonnet 으로 변경해주세요.
    response = bedrock.invoke_model(
        body=json.dumps(body),
        #modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"  # Claude 3.5 Sonnet
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",      # Claude 3 Sonnet
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

## 2. Claude 3.x 모델 호출하여 이미지 pdf 처리

In [19]:
pdf_path = "./rag_data/Steel Business Briefing_04 Jul 2024.pdf"
images = pdf_to_images(pdf_path)

for i, image in enumerate(images):
    image_base64 = image_to_base64(image)
    response = get_claude_response(image_base64)
    print(f"Page {i+1} Summary:")
    print(response)
    print("\n---\n")

Page 1 Summary:
이 이미지는 SBB Global의 Commodity Insights 부서에서 발행한 SBB Daily Briefing의 표지 부분입니다. 상단에는 Platts Connect에서도 디지털 버전을 이용할 수 있다는 안내가 있습니다.

Price snapshot 섹션에는 여러 원자재의 일일 가격과 주간 가격 정보가 테이블 형태로 제시되어 있습니다. 상품 기호, 단위, 가격, 변동분, 변동률, 평가 일자 등의 세부 정보가 포함되어 있습니다.

Contents 섹션에서는 이 브리핑의 주요 내용들이 지역별로 간략히 소개되어 있습니다. 아시아, 아메리카, EMEA, CIS 등의 지역과 관련된 원자재 시장 뉴스와 업데이트 사항들이 요약되어 있습니다.

전반적으로 이 표지는 SBB Daily Briefing의 개요와 주요 내용, 그리고 가격 정보를 한눈에 보여주는 역할을 하고 있습니다.

---

Page 2 Summary:
이 문서에는 아시아 지역의 평판 제품 시장 동향에 대한 정보가 포함되어 있습니다. 주요 내용은 다음과 같습니다:

1. Asia 섹션
- 중국에서 가격은 전반적으로 높은 수준을 유지했지만, 후강 지역 교역소는 3.85/tmt 하락했고 연운항 교역소는 3tmt, 광주 교역소는 0.72% 하락했습니다.
- 동남아시아 지역 제품 가격은 변동 없이 전일 수준을 유지했습니다.
- 한국 HRC 시장은 정체 상태였고 구매 관심은 약 $10/mt 정도였습니다. 

2. Flat Products 섹션 
- 일본과 한국 제품 가격은 전일 대비 변동 없었습니다.
- 평가서는 총 $2,500/mt HRC의 5월 물량 수주를 전했습니다.

3. Southeast Asia 섹션
- 수출 가격 변동 없었고 $620/t CFR 중국 및 동남아시아 지역 수준이었습니다.

4. Asian HRC at standstill; buying indications lagging behind
- 구매 관심이 약 $10/mt 수준이었습니다.

5. Indian rebar